# **JPM MLCOE TSRL 2026 Q1**
---
**Heartie CHEN**

## 0. Global Configurations
---

In [1]:
from pathlib import Path
import re
import random
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from typing import Tuple


pd.set_option('display.float_format', lambda v: f"{v:,.2f}")


print('yfinance:', yf.__version__)
print('pandas:', pd.__version__)
print('tensorflow:', tf.__version__)

ModuleNotFoundError: No module named 'yfinance'

In [ ]:
SEED = 3
random.seed(SEED)
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

DATA_DIR = Path("..").resolve() / "data" / "yfinance" # "sec" / "yfinance"
DATA_DIR.mkdir(parents = True, exist_ok = True)
TARGET_FREQ = 'yearly' # 'quarterly' / 'yearly'

## 1. Data Pipeline
---

### 1.1 Column Aliases and Identity Checks

In [ ]:
BS_ALIASES = {
    "Total Assets": ["Total Assets", "TotalAssets"],
    "Total Liabilities": [
        "Total Liab", "Total Liabilities", "Total Liabilities Net Minority Interest",
        "TotalLiabilitiesNetMinorityInterest", "TotalLiabilities"
    ],
    "Total Equity": [
        "Total Stockholder Equity", "Total Equity Gross Minority Interest",
        "Stockholders Equity", "TotalEquityGrossMinorityInterest", "StockholdersEquity"
    ],
    "Accounts Receivable": ["Accounts Receivable", "AccountsReceivable"],
    "Accounts Payable": ["Accounts Payable", "AccountsPayable"],
    "Net PPE": ["Net PPE", "NetPPE"],
    "Retained Earnings": ["Retained Earnings", "RetainedEarnings"],
    "Inventory": ["Inventory", "Inventories"],
}


def canonicalize_bs(df: pd.DataFrame) -> pd.DataFrame:
    """Return a balance sheet with canonical column names."""

    rename_map = {}

    for canon, candidates in BS_ALIASES.items():
        for c in candidates:

            if c in df.columns:
                rename_map[c] = canon
                
                break


    return df.rename(columns = rename_map)


def identity_residual(df: pd.DataFrame) -> pd.Series:
    """Compute balance sheet identity residuals."""

    if "Total Liabilities" not in df.columns or "Total Equity" not in df.columns:
        df = canonicalize_bs(df)
    
    required = ["Total Assets", "Total Liabilities", "Total Equity"]
    missing = [c for c in required if c not in df]

    if missing:

        raise KeyError(f"Missing columns: {missing}")


    return df["Total Assets"] - (df["Total Liabilities"] + df["Total Equity"])


def summarize_identity(resid: pd.Series) -> pd.Series:
    """Summarize residual statistics."""

    return pd.Series({

        "mean": resid.mean(),
        "std": resid.std(),
        "max_abs": resid.abs().max(),
    })

### 1.2 Load Data with Caching

In [ ]:
SPECIAL_TICKERS = {
    "700": "0700.HK",
    "1810": "1810.HK",
    "9633": "9633.HK",
    "9987": "9987.HK",
    "9988": "9988.HK",
    "BRK B": "BRK-B",
    "NESN": "NESN.SW",
}


def normalize_ticker(raw: str) -> str:
    """Normalize tickers and apply project-specific mappings."""

    value = raw.strip()
    upper = value.upper()

    if upper in SPECIAL_TICKERS:

        return SPECIAL_TICKERS[upper]

    if re.fullmatch(r"\d+", value):

        return value.zfill(4) + ".HK"


    return upper


def slugify(ticker: str) -> str:
    """Create a filesystem-friendly slug for a ticker."""

    return re.sub(r"[^a-z0-9]+", "_", ticker.lower()).strip("_")


def _fetch_statements(tkr, freq: str):
    """Fetch statements from Yahoo Finance for a ticker."""

    bs = tkr.get_balance_sheet(freq = freq)
    is_df = tkr.get_financials(freq = freq)

    if bs is None or is_df is None:

        return pd.DataFrame(), pd.DataFrame()

    bs = bs.T.sort_index()
    is_df = is_df.T.sort_index()


    return bs, is_df


def _load_cached(path: Path) -> pd.DataFrame:
    """Load a cached statement CSV if available."""

    return pd.read_csv(path, index_col = 0, parse_dates = True)


def load_statements(ticker: str, freq: str = TARGET_FREQ):
    """Load or fetch statements for a ticker and frequency."""

    ticker = normalize_ticker(ticker)
    slug = slugify(ticker)
    freq = freq.lower()

    data_dir = DATA_DIR / freq
    data_dir.mkdir(parents = True, exist_ok = True)

    bs_path = data_dir / f"{slug}_balance_sheet_{freq}.csv"
    is_path = data_dir / f"{slug}_income_statement_{freq}.csv"

    if bs_path.exists() and is_path.exists():
        bs = _load_cached(bs_path)
        is_df = _load_cached(is_path)

    else:
        tkr = yf.Ticker(ticker)
        bs, is_df = _fetch_statements(tkr, freq)

        if not bs.empty and not is_df.empty:
            bs.to_csv(bs_path)
            is_df.to_csv(is_path)

    if bs.empty or is_df.empty:

        raise RuntimeError(f"Failed to fetch statements for {ticker} ({freq})")

    bs = canonicalize_bs(bs)


    return bs, is_df, freq


TICKERS = [
    "AAPL", "GOOG", "700", "1810", "IBM", "TSLA",
    "9633", "9987", "9988", "IBKR", "KO", "MCD",
    "EL", "BRK B", "NESN"
]

PRIMARY_TICKER = TICKERS[0]
bs, is_df, stmt_freq = load_statements(PRIMARY_TICKER, freq = TARGET_FREQ)


print("Loaded tickers:", [normalize_ticker(t) for t in TICKERS])
print("Statement frequency:", stmt_freq)
print("Balance sheet shape:", bs.shape)
print("Income statement shape:", is_df.shape)
print("Identity residual stats:", summarize_identity(identity_residual(bs)))
bs.head()

## 2. Features & Dataset

### 2.1 Feature Engineering

#### 2.1.1 Derived Drivers (DSO/DPO/DIH, Margins, Growth, Logs)

In [ ]:
def _pick(df: pd.DataFrame, options):
    """Pick the first available series from the provided options."""

    for c in options:

        if c in df.columns:

            return df[c]

    raise KeyError(f'Missing columns: {options}')


def compute_features(bs: pd.DataFrame, is_df: pd.DataFrame, days: float = 365.0, growth_periods: int = 1) -> pd.DataFrame:
    """Compute financial ratios and growth features."""

    rev = _pick(is_df, ['Total Revenue', 'Operating Revenue', 'TotalRevenue', 'OperatingRevenue', 'Revenues'])
    cogs = _pick(is_df, ['Cost Of Revenue', 'Cost of Revenue', 'CostOfRevenue', 'ReconciledCostOfRevenue'])
    op_inc = _pick(is_df, ['Operating Income', 'OperatingIncome'])
    net_inc = _pick(is_df, ['Net Income', 'NetIncome', 'NetIncomeFromContinuingOperationNetMinorityInterest'])
    ar = _pick(bs, ['Accounts Receivable', 'AccountsReceivable'])
    ap = _pick(bs, ['Accounts Payable', 'AccountsPayable'])
    inv = _pick(bs, ['Inventory', 'Inventories'])

    feats = pd.DataFrame(index = bs.index)

    sales_per_day = rev / days
    cogs_per_day = cogs / days

    feats['dso'] = ar / sales_per_day
    feats['dpo'] = ap / cogs_per_day
    feats['dih'] = inv / cogs_per_day
    feats['gross_margin'] = (rev - cogs) / rev
    feats['op_margin'] = op_inc / rev
    feats['net_margin'] = net_inc / rev
    feats['rev_yoy'] = rev.pct_change(periods = growth_periods)
    feats['cogs_yoy'] = cogs.pct_change(periods = growth_periods)
    feats['netinc_yoy'] = net_inc.pct_change(periods = growth_periods)
    feats['log_rev'] = np.log1p(rev)
    feats['log_assets'] = np.log1p(_pick(bs, ['Total Assets', 'TotalAssets']))
    
    feats = feats.replace([np.inf, -np.inf], np.nan)
    feats = feats.sort_index().ffill().bfill()


    return feats


period_days = 365.0 if TARGET_FREQ == 'yearly' else 90.0
growth_periods = 1 if TARGET_FREQ == 'yearly' else 4

features = compute_features(bs, is_df, days = period_days, growth_periods = growth_periods)
features.tail()

### 2.2 Dataset Assembly

In [ ]:
TARGET_BS = ['Total Assets', 'Total Liabilities', 'Total Equity']
TARGET_LE = ['Total Liabilities', 'Total Equity']
NET_INCOME_COL = 'Net Income'
STATE_COLS = [
    'Accounts Receivable', 'Accounts Payable', 'Inventory', 'Net PPE',
    'Total Liabilities', 'Total Equity', 'Retained Earnings'
]


def _has_any(df: pd.DataFrame, options: list[str]) -> bool:
    """Return True if any candidate column exists."""

    return any(c in df.columns for c in options)


def missing_required_columns(bs: pd.DataFrame, is_df: pd.DataFrame) -> list[str]:
    """Return missing required columns for BS and IS."""

    missing = []

    if not _has_any(bs, ['Total Assets', 'TotalAssets']):
        missing.append('Total Assets')
    
    if not _has_any(bs, ['Total Liabilities', 'Total Liabilities', 'TotalLiabilities', 'Total Liabilities Net Minority Interest', 'TotalLiabilitiesNetMinorityInterest']):
        missing.append('Total Liabilities')
    
    if not _has_any(bs, ['Total Equity', 'Stockholders Equity', 'StockholdersEquity', 'Total Equity Gross Minority Interest', 'TotalEquityGrossMinorityInterest']):
        missing.append('Total Equity')
    
    if not _has_any(bs, ['Accounts Receivable', 'AccountsReceivable']):
        missing.append('Accounts Receivable')
    
    if not _has_any(bs, ['Accounts Payable', 'AccountsPayable']):
        missing.append('Accounts Payable')
    
    if not _has_any(bs, ['Inventory', 'Inventories']):
        missing.append('Inventory')
    
    if not _has_any(bs, ['Net PPE', 'NetPPE']):
        missing.append('Net PPE')
    
    if not _has_any(bs, ['Retained Earnings', 'RetainedEarnings']):
        missing.append('Retained Earnings')

    if not _has_any(is_df, ['Total Revenue', 'Operating Revenue', 'TotalRevenue', 'OperatingRevenue', 'Revenues']):
        missing.append('Total Revenue')
   
    if not _has_any(is_df, ['Cost Of Revenue', 'Cost of Revenue', 'CostOfRevenue', 'ReconciledCostOfRevenue']):
        missing.append('Cost Of Revenue')
    
    if not _has_any(is_df, ['Operating Income', 'OperatingIncome']):
        missing.append('Operating Income')
    
    if not _has_any(is_df, ['Net Income', 'NetIncome', 'NetIncomeFromContinuingOperationNetMinorityInterest']):
        missing.append('Net Income')


    return missing


def build_aligned_for_ticker(ticker: str) -> pd.DataFrame:
    """Build an aligned dataset for one ticker."""

    bs, is_df, freq = load_statements(ticker, freq = TARGET_FREQ)
    missing = missing_required_columns(bs, is_df)
    
    if missing:

        print(f"{ticker}: skip, missing {missing}")

        return pd.DataFrame()

    period_days = 365.0 if freq == 'yearly' else 90.0
    growth_periods = 1 if freq == 'yearly' else 4

    features = compute_features(bs, is_df, days = period_days, growth_periods = growth_periods)
    targets = bs[TARGET_BS].copy()
    net_income = _pick(is_df, ['Net Income', 'NetIncome', 'NetIncomeFromContinuingOperationNetMinorityInterest']).rename(NET_INCOME_COL)
    net_income = net_income.reindex(targets.index)

    # Align features, targets, and net income on shared dates.
    dataset = features.join(targets, how = 'inner').join(net_income, how = 'inner')
    dataset = dataset.dropna()

    rev_series = _pick(is_df, ['Total Revenue', 'Operating Revenue', 'TotalRevenue', 'OperatingRevenue', 'Revenues'])
    # Shift state variables to create t-1 inputs.
    prev_state_df = bs.reindex(columns = STATE_COLS).shift(1)
    prev_state_df.columns = [f'prev_{c}' for c in STATE_COLS]
    prev_state_df['prev_Total Revenue'] = rev_series.reindex(targets.index).shift(1)

    # Combine current data with lagged state for modeling.
    aligned = dataset.join(prev_state_df, how = 'inner').dropna()
    aligned['ticker'] = ticker
    aligned['stmt_freq'] = freq


    return aligned


aligned_list = []

for ticker in [normalize_ticker(t) for t in TICKERS]:

    try:
        aligned_t = build_aligned_for_ticker(ticker)

        if aligned_t.empty:

            print(f"{ticker}: empty after alignment")

        else:

            print(f"{ticker}: aligned rows = {aligned_t.shape[0]}")
            aligned_list.append(aligned_t)

    except Exception as exc:

        print(f"{ticker}: failed ({exc})")

if not aligned_list:

    raise RuntimeError('No aligned data available')

aligned = pd.concat(aligned_list, axis = 0).sort_index()
prev_cols = [f'prev_{c}' for c in STATE_COLS] + ['prev_Total Revenue']
drop_cols = TARGET_BS + [NET_INCOME_COL] + prev_cols + ['ticker', 'stmt_freq']
FEATURE_COLS = [c for c in aligned.columns if c not in drop_cols]


print('Dataset shape:', aligned.shape)
print('Feature cols:', len(FEATURE_COLS), 'Target cols:', len(TARGET_LE) + 1)
print('Tickers:', aligned['ticker'].nunique())

### 2.3 Prev-state Matrix for Algebraic Layer

In [ ]:
X_feat = aligned[FEATURE_COLS].values.astype('float32')
Y_bs = aligned[TARGET_LE].values.astype('float32')
Y_earn = aligned[[NET_INCOME_COL]].values.astype('float32')
X_prev = aligned[prev_cols].values.astype('float32')


print('Aligned shapes:', X_feat.shape, X_prev.shape, Y_bs.shape, Y_earn.shape)

### 2.4 Scaling (z-score) for Stability

In [ ]:
feat_scaler = StandardScaler()
bs_scaler = StandardScaler()
earn_scaler = StandardScaler()
prev_scaler = StandardScaler()

X_feat_scaled = feat_scaler.fit_transform(X_feat)
Y_bs_scaled = bs_scaler.fit_transform(Y_bs)
Y_earn_scaled = earn_scaler.fit_transform(Y_earn)
X_prev_scaled = prev_scaler.fit_transform(X_prev)

### 2.5 Train/Val Split on Scaled Data

In [ ]:
n = X_feat_scaled.shape[0]
train_size = max(1, int(0.8 * n))

X_train_feat = X_feat_scaled[:train_size]
Y_train_bs = Y_bs_scaled[:train_size]
Y_train_earn = Y_earn_scaled[:train_size]
X_train_prev = X_prev_scaled[:train_size]

X_val_feat = X_feat_scaled[train_size:] if train_size < n else X_feat_scaled[train_size - 1:]
Y_val_bs = Y_bs_scaled[train_size:] if train_size < n else Y_bs_scaled[train_size - 1:]
Y_val_earn = Y_earn_scaled[train_size:] if train_size < n else Y_earn_scaled[train_size - 1:]
X_val_prev = X_prev_scaled[train_size:] if train_size < n else X_prev_scaled[train_size - 1:]


print('Scaled train:', X_train_feat.shape, Y_train_bs.shape, Y_train_earn.shape)
print('Scaled val:', X_val_feat.shape, Y_val_bs.shape, Y_val_earn.shape)

## 3. TensorFlow (Pareja/Pelaez Constrained)

### 3.1 TF Model with Algebraic Generator + Earnings Head

In [ ]:
class AlgebraicBS(keras.layers.Layer):
    """Layer that enforces algebraic balance sheet constraints."""

    def __init__(self):
        """Initialize sublayers for balance sheet generation."""
        
        super().__init__()
        
        self.hidden = keras.layers.Dense(64, activation = 'relu')
        self.rev_head = keras.layers.Dense(1, activation = 'relu')
        self.cogs_head = keras.layers.Dense(1, activation = 'relu')
        self.drivers = keras.layers.Dense(5)
        self.margin_head = keras.layers.Dense(1)
        self.payout_head = keras.layers.Dense(1)
        self.earn_head = keras.layers.Dense(1, name = 'net_income_head')


    def call(self, inputs: Tuple[tf.Tensor, tf.Tensor]):
        """Compute constrained balance sheet and earnings outputs."""

        if not isinstance(inputs, (tuple, list)):

            raise TypeError("inputs must be (features, prev_state)")

        features, prev_state = inputs
        ar_prev, ap_prev, inv_prev, ppe_prev, liab_prev, equity_prev, re_prev, rev_prev = tf.split(prev_state, num_or_size_splits = 8, axis = -1)
        hidden = self.hidden(features)
        rev_predicate = self.rev_head(hidden)
        cogs_predicate = self.cogs_head(hidden)
        drivers_raw = self.drivers(hidden)

        dso = tf.nn.softplus(drivers_raw[:, 0:1])
        dpo = tf.nn.softplus(drivers_raw[:, 1:2])
        dih = tf.nn.softplus(drivers_raw[:, 2:3])
        dep_rate = tf.nn.sigmoid(drivers_raw[:, 3:4]) * 0.2
        capex_rate = tf.nn.sigmoid(drivers_raw[:, 4:5]) * 0.2
        net_margin = tf.tanh(self.margin_head(hidden)) * 0.5
        div_payout = tf.nn.sigmoid(self.payout_head(hidden))

        sales_per_day = rev_predicate / 365.0
        cogs_per_day = cogs_predicate / 365.0

        ar_next = dso * sales_per_day
        ap_next = dpo * cogs_per_day
        inv_next = dih * cogs_per_day
        dep = dep_rate * ppe_prev
        capex = capex_rate * rev_predicate
        ppe_next = ppe_prev + capex - dep
        net_income = net_margin * rev_predicate
        earn_predicate = self.earn_head(hidden)
        div = div_payout * net_income
        re_next = re_prev + net_income - div

        other_equity = tf.nn.relu(equity_prev - re_prev)

        equity_next = re_next + other_equity

        other_liab_prev = tf.nn.relu(liab_prev - ap_prev)
        growth = tf.where(rev_prev > 0, rev_predicate / rev_prev - 1.0, tf.zeros_like(rev_predicate))

        other_liab_next = other_liab_prev * (1.0 + growth)
        liab_next = ap_next + other_liab_next
        assets_wo_cash = ar_next + inv_next + ppe_next
        cash_next = equity_next + liab_next - assets_wo_cash
        assets_next = liab_next + equity_next
        bs_out = tf.concat([liab_next, equity_next], axis = -1)


        return bs_out, earn_predicate


def build_pareja_model(feat_dim: int, state_dim: int = 8):
    """Build a Pareja/Pelaez-constrained model."""

    feat_in = keras.Input(shape = (feat_dim,), name = 'features')
    state_in = keras.Input(shape = (state_dim,), name = 'prev_state')
    bs_out, earn_out = AlgebraicBS()([feat_in, state_in])
    
    
    return keras.Model([feat_in, state_in], [bs_out, earn_out], name = 'bs_pareja_style')


pareja_model = build_pareja_model(feat_dim = X_feat_scaled.shape[1], state_dim = X_prev_scaled.shape[1])
pareja_model.summary()

### 3.2 Train/Evaluate (MAE on BS + Earnings)

In [ ]:
pareja_model.compile(
    optimizer = keras.optimizers.Adam(1e-3),
    loss = [keras.losses.MeanAbsoluteError(), keras.losses.MeanAbsoluteError()],
    loss_weights = [1.0, 0.3],
)

hist = pareja_model.fit(
    [X_train_feat, X_train_prev], [Y_train_bs, Y_train_earn],
    
    validation_data = (
        [X_val_feat, X_val_prev], [Y_val_bs, Y_val_earn]
    ) if len(X_val_feat) > 0 else None,
    
    epochs = 20,
    batch_size = 2,
    verbose = 0,
)


print('Final Train Losses:', '\n', {k: v[-1] for k, v in hist.history.items() if 'loss' in k}, '\n')

bs_predicate_scaled, earn_predicate_scaled = pareja_model.predict([X_feat_scaled, X_prev_scaled], verbose = 0)
bs_predicate = bs_scaler.inverse_transform(bs_predicate_scaled)
earn_predicate = earn_scaler.inverse_transform(earn_predicate_scaled)

In [ ]:
bs_predicate_df = pd.DataFrame(bs_predicate, columns = TARGET_LE, index = aligned.index)
assets_predicate = bs_predicate_df['Total Liabilities'] + bs_predicate_df['Total Equity']
bs_predicate_full = bs_predicate_df.copy()
bs_predicate_full['Total Assets'] = assets_predicate
bs_predicate_full = bs_predicate_full[TARGET_BS]
resid_predicate = bs_predicate_full['Total Assets'] - (bs_predicate_full['Total Liabilities'] + bs_predicate_full['Total Equity'])

pred_full = bs_predicate_full.copy()
pred_full['ticker'] = aligned['ticker'].values
pred_full['stmt_freq'] = aligned['stmt_freq'].values
pred_full['pred_net_income'] = earn_predicate.flatten()
pred_full['pred_resid'] = pred_full['Total Assets'] - (pred_full['Total Liabilities'] + pred_full['Total Equity'])

actual_bs = aligned[TARGET_BS].copy()
actual_bs.columns = [f'actual_{c}' for c in actual_bs.columns]
pred_full = pd.concat([pred_full, actual_bs], axis = 1)
pred_full['actual_net_income'] = aligned[NET_INCOME_COL].values

pred_full['err_assets'] = pred_full['Total Assets'] - pred_full['actual_Total Assets']
pred_full['err_liab'] = pred_full['Total Liabilities'] - pred_full['actual_Total Liabilities']
pred_full['err_equity'] = pred_full['Total Equity'] - pred_full['actual_Total Equity']
pred_full['err_net_income'] = pred_full['pred_net_income'] - pred_full['actual_net_income']
pred_full['abs_err_assets'] = pred_full['err_assets'].abs()
pred_full['abs_err_liab'] = pred_full['err_liab'].abs()
pred_full['abs_err_equity'] = pred_full['err_equity'].abs()
pred_full['abs_err_net_income'] = pred_full['err_net_income'].abs()

summary = pred_full.groupby('ticker').agg(
    samples = ('pred_net_income', 'size'),
    mean_pred_assets = ('Total Assets', 'mean'),
    mean_pred_liab = ('Total Liabilities', 'mean'),
    mean_pred_equity = ('Total Equity', 'mean'),
    mean_pred_net_income = ('pred_net_income', 'mean'),
    mean_actual_assets = ('actual_Total Assets', 'mean'),
    mean_actual_liab = ('actual_Total Liabilities', 'mean'),
    mean_actual_equity = ('actual_Total Equity', 'mean'),
    mean_actual_net_income = ('actual_net_income', 'mean'),
    mae_assets = ('abs_err_assets', 'mean'),
    mae_liab = ('abs_err_liab', 'mean'),
    mae_equity = ('abs_err_equity', 'mean'),
    mae_net_income = ('abs_err_net_income', 'mean'),
)

def format_df_for_view(df: pd.DataFrame, decimals: int = 2) -> pd.DataFrame:
    """Create a display-only view with formatted numerics."""

    view = df.copy()
    num_cols = view.select_dtypes(include = [np.number]).columns
    fmt = f"{{:,.{decimals}f}}"
    view[num_cols] = view[num_cols].map(lambda v: "" if pd.isna(v) else fmt.format(v))

    return view


bs_predicate_full_view = format_df_for_view(bs_predicate_full)
pred_full_view = format_df_for_view(pred_full)
summary_view = format_df_for_view(summary)

earn_head_str = np.array2string(
    earn_predicate.flatten(),
    formatter = {"float_kind": lambda v: f"{v:,.2f}"}
)